## startup	
* 老版脚本，用于处理meta

In [1]:
import cooler
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import glob
from scipy.stats import mannwhitneyu
from mpl_toolkits.axes_grid1 import make_axes_locatable
import bioframe
import matplotlib.pyplot as plt
from matplotlib import font_manager
import cooltools
from cooltools import insulation
from skimage.filters import threshold_li, threshold_otsu
import bioframe
import bbi
import pyBigWig
import hicstraw
from sklearn.isotonic import IsotonicRegression
font_manager.fontManager.ttflist.append(font_manager.FontProperties(fname='/cluster/home/futing/miniforge-pypy3/envs/hic/fonts/Arial.ttf'))
arial_font = font_manager.FontProperties(fname='Arial.ttf')
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Arial'] 
plt.rcParams['pdf.fonttype'] = 42



In [117]:
loop=pd.read_csv('/cluster/home/futing/Project/GBM/HiC/10loop/consensus/result/subtype/subtype_promoter.bedpe',sep='\t')
RNA=pd.read_csv('/cluster/home/futing/ref_genome/hg38_gencode/humanGTF',sep='\t',header=None)
#gene=pd.read_csv('/cluster/home/futing/ref_genome/hg38_gencode/GRCh38.promoter.bed',sep='\t',header=None)
gene=pd.read_csv('/cluster/home/futing/ref_genome/hg38_gencode/GRCh38.promoter_nodot.bed',sep='\t')

GSC=pd.read_csv('/cluster/home/futing/Project/GBM/RNA/GSC28_TPM.txt',sep=' ')
#EGA=pd.read_csv('/cluster/home/tmp/EGA/19to38/gene-TPM-EGA100.txt',sep='\t')
EGA=pd.read_csv('/cluster/home/futing/Project/GBM/RNA/merge/EGA_tpm.txt',sep='\t',)
other=pd.read_csv('/cluster/home/futing/Project/GBM/RNA/merge/0830-tpm-matrix.txt',sep='\t')
#other=pd.read_csv('/cluster/home/futing/Project/GBM/RNA/merge/other.txt',sep='\t')

In [48]:
#EGA.columns=EGA.columns.str.split('.tophat.genes.results').str[0].str.split('.accepted_hits.genes.results').str[0].str.split('Tophat_').str[-1]

EGA.columns= EGA.columns.map(lambda x: x if '.SF' in x else x.replace('SF', '.SF', 1))

EGA.to_csv('/cluster/home/futing/Project/GBM/RNA/merge/EGA_tpm.txt',sep='\t',index=False)

In [123]:
# 用meta文件对列名进行重命名
meta=pd.read_csv('/cluster/home/futing/Project/GBM/RNA/merge/metadata_s.tsv',sep='\t')
meta.columns=['srr','id','path','sample']

In [124]:


meta['id'] = meta['id'].fillna('').astype(str)
# 找出重复值
duplicates = meta['id'].duplicated(keep=False)

# 仅对重复值添加后缀
meta.loc[duplicates, 'sample'] =  meta['id'] +'_' +(
    meta.loc[duplicates].groupby('id').cumcount() + 1
).astype(str)
#.radd(meta['id'] + '_')
column_mapping = dict(zip(meta['srr'], meta['sample']))
other.rename(columns=column_mapping,inplace=True)

#meta.to_csv('/cluster/home/futing/Project/GBM/RNA/merge/metadata_s.tsv',sep='\t',index=False)


In [15]:
gene=pd.read_csv('/cluster/home/futing/ref_genome/hg38_gencode/genebed/gencode.v43.gene.bed',sep='\t',header=None)
gene=gene.iloc[:,[0,1,2,3,6,7,8]]
gene.columns=['chr','start','end','ENSG','symbol','type','strand']
gene['ENTREZ']=gene['ENSG'].str.replace(r'\.\d+(_PAR_Y)?', r'\1', regex=True)
#gene=gene.iloc[:,[0,1,2,7,4,5,6]]
gene.to_csv('/cluster/home/futing/ref_genome/hg38_gencode/genebed/gencode.v43.gene_nodot.bed',sep='\t',index=False)

In [12]:
gene=pd.read_csv('/cluster/home/futing/ref_genome/hg38_gencode/genebed/gencode.v43.gene_nodot.bed',sep='\t',header=None)

## 处理count的基因名

In [129]:
EGA['ENTREZ']=EGA['gene'].str.split('.').str[0]
EGA=EGA.groupby('ENTREZ').sum()
EGA.reset_index(inplace=True)
EGA.drop(columns=['gene'],inplace=True)


In [128]:
GSC.reset_index(inplace=True)
GSC.rename(columns={'index':'ENTREZ'},inplace=True)

In [ ]:
other['ENTREZ']=other['gene'].str.split('.').str[0]
other=other.groupby('ENTREZ').sum()
other.reset_index(inplace=True)
other.drop(columns=['gene'],inplace=True)

In [79]:
GSC.shape,EGA.shape,other.shape

((58220, 29), (60664, 101), (63450, 48))

In [131]:
all=pd.merge(GSC,EGA,on='ENTREZ',how='outer')
all=pd.merge(all,other,on='ENTREZ',how='outer')

In [8]:
#all.to_csv('/cluster/home/futing/Project/GBM/RNA/merge/all.txt',sep='\t',index=False)
all=pd.read_csv('/cluster/home/futing/Project/GBM/RNA/merge/all.txt',sep='\t')

In [10]:
all.index

Index(['ENSG00000000003', 'ENSG00000000005', 'ENSG00000000419',
       'ENSG00000000457', 'ENSG00000000460', 'ENSG00000000938',
       'ENSG00000000971', 'ENSG00000001036', 'ENSG00000001084',
       'ENSG00000001167',
       ...
       'ENSG00000293587', 'ENSG00000293588', 'ENSG00000293591',
       'ENSG00000293592', 'ENSG00000293594', 'ENSG00000293595',
       'ENSG00000293596', 'ENSG00000293597', 'ENSG00000293599',
       'ENSG00000293600'],
      dtype='object', length=63497)

In [ ]:
all=pd.merge(all,gene,on='ENTREZ',how='outer')
all.to_csv('/cluster/home/futing/Project/GBM/RNA/merge/all_anno.txt',sep='\t',index=False)

## filter loops

In [30]:
all=pd.read_csv('/cluster/home/futing/Project/GBM/RNA/merge/all_anno.txt',sep='\t')
#subtype=pd.read_csv('/cluster/home/futing/Project/GBM/HiC/10loop/consensus/result/subtype/subtype_promoter2.bedpe',sep='\t')
subtype=pd.read_csv('/cluster/home/futing/Project/GBM/HiC/10loop/consensus/result/ctrl/all_promotor.bedpe',sep='\t')

In [31]:
subtype.head()

,chr,start,end,chr.1,start.1,end.1,astro,NPC,iPSC,classical,mesenchymal,proneural,neural,num,chr.2,start.2,end.2,ENTREZ,symbol,type
0,chr1,1740000,1770000,chr1,1810000,1840000,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1,chr1,1739056,1740556,ENSG00000215790,SLC35E2A,transcribed_unprocessed_pseudogene
1,chr1,1740000,1770000,chr1,1810000,1840000,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1,chr1,1745491,1746991,ENSG00000290854,ENSG00000290854,lncRNA
2,chr1,1790000,1820000,chr1,1880000,1910000,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1,chr1,1890222,1891722,ENSG00000231050,GNB1-DT,lncRNA
3,chr1,1790000,1820000,chr1,1880000,1910000,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1,chr1,1891791,1893291,ENSG00000078369,GNB1,protein_coding
4,chr1,1790000,1820000,chr1,1890000,1920000,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1,chr1,1890222,1891722,ENSG00000231050,GNB1-DT,lncRNA


In [ ]:
# 找到不同loop调控同一个基因的基因
dupgene=subtype.loc[subtype['symbol'].duplicated(),:]
dupgene_p2=subtype.loc[subtype['num']>1,:]

dupgene=pd.concat([dupgene,dupgene_p2])
dupgene=dupgene.sort_values('symbol')

In [6]:
dupprogene=dupgene.loc[dupgene['type']=='protein_coding',:]

In [7]:
dupprogene.loc[:,'start']=dupprogene['start']+30000
dupprogene.loc[:,'end']=dupprogene['end']-30000
dupprogene.loc[:,'start.1']=dupprogene['start.1']+30000
dupprogene.loc[:,'end.1']=dupprogene['end.1']-30000

In [8]:
dupprogene.to_csv('/cluster/home/futing/Project/GBM/HiC/10loop/consensus/result/subtype/subtype_produpgene2.bedpe',sep='\t',index=False)

In [ ]:
#这些都是每个亚型特异性的基因
subtypelist=['classical','mesenchymal','neural','proneural','astro','NPC','iPSC']

df=pd.DataFrame()
for i in subtypelist:
    gene=subtype.loc[(subtype[i]==1) & (subtype['num']==1),'ENTREZ']
    df[i]=pd.Series(gene.values)

# 输入到R里与limma分析的基因进行比较   

In [36]:
df.head()

,classical,mesenchymal,neural,proneural,astro,NPC,iPSC
0,ENSG00000182873,ENSG00000169885,ENSG00000196581,ENSG00000225196,ENSG00000215790,ENSG00000142611,ENSG00000231050
1,ENSG00000157911,ENSG00000178821,ENSG00000228750,ENSG00000252969,ENSG00000290854,ENSG00000177133,ENSG00000078369
2,ENSG00000149527,ENSG00000182873,ENSG00000237436,ENSG00000048707,ENSG00000231050,ENSG00000142611,ENSG00000169885
3,ENSG00000238164,ENSG00000272235,ENSG00000171735,ENSG00000272482,ENSG00000078369,ENSG00000177133,ENSG00000178821
4,ENSG00000225931,ENSG00000287828,ENSG00000284747,ENSG00000204518,ENSG00000231050,ENSG00000142611,ENSG00000157911


In [37]:
df.to_csv('/cluster/home/futing/Project/GBM/HiC/10loop/consensus/result/subtype/subtype_ENSG.txt',sep='\t',index=False)